In [ ]:
import psycopg2
from enum import Enum

class DatabaseConnector:
    def __init__(self, db_name, db_user):
        self.db_name = db_name
        self.db_user = db_user
    
    def __enter__(self):
        self.conn = psycopg2.connect(f"dbname={self.db_name} user={self.db_user} password=data")
        return self.conn

    def __exit__(self, type, value, tb):
        self.conn.close()

class EntityOptions(Enum):
    SGFB = 'Slow Growth Fast Bust'
    V = 'Volatile'
    FD = 'Fluctuating Decline'
    FR = 'Fluctuating Rise'

class EntitiesValuesFunctions:
    def __init__(self, databaseConnection):
        self.conn = databaseConnection

    def get_existing_entities(self):
        entities = []
        try:
            with self.conn:
                with self.conn.cursor() as cur:
                    cur.callproc('get_existing_entities')
                    entities = cur.fetchall()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            return entities

    def add_entity(self, code, option, firstConstant, secondConstant, thirdConstant):
        success = False
        try:
            with self.conn:
                with self.conn.cursor() as cur:
                    cur.callproc('add_entity', (code, option, firstConstant, secondConstant, thirdConstant))
                    success = cur.fetchall()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            return success

    def add_entity_value(self, code, timestamp, value):
        success = False
        try:
            with self.conn:
                with self.conn.cursor() as cur:
                    cur.callproc('add_entity_value', (code, timestamp, value))
                    success = cur.fetchall()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            return success

    def get_entity_details(self, code):
        entityDetails = []
        try:
            with self.conn:
                with self.conn.cursor() as cur:
                    cur.callproc('get_entity_details', (code,))
                    entityDetails = cur.fetchall()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            return entityDetails

    def get_values(self, code, timestamp):
        values = []
        try:
            with self.conn:
                with self.conn.cursor() as cur:
                    cur.callproc('get_values', (code, timestamp))
                    values = cur.fetchall()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            return values

In [ ]:
import pytest
from packages.Databases import DatabaseConnector, EntityOptions, EntitiesValuesFunctions
from datetime import datetime, timedelta

class EntitiesValuesTests:
    with DatabaseConnector('EntitiesAndValues', 'data_seeder') as conn:
        _entitiesValues = EntitiesValuesFunctions(conn)

        def test_add_entity(_entitiesValues):
            assert _entitiesValues.add_entity('AAAAA', EntityOptions.SGFB.value, 0.2, 0.1, 0.5)

        def test_get_entities(_entitiesValues):
            assert 'AAAAA' in _entitiesValues.get_existing_entities()

        def test_get_entity_details(_entitiesValues):
            assert ('AAAAA', EntityOptions.SGFB.value, 0.2, 0.1, 0.5) == _entitiesValues.get_entity_details('AAAAA')

        def test_add_entity_value(_entitiesValues):
            assert _entitiesValues.add_entity_value('AAAAA', datetime.now(), 7.2)

        def test_get_values(_entitiesValues):
            values = _entitiesValues.get_values('AAAAA', datetime.now() - timedelta(days=1))
            assert values[0] == 'AAAAA'
            assert values[1].year == datetime.now().year
            assert values[1].month == datetime.now().month
            assert values[1].day == datetime.now().day
            assert values[2] == 7.2

In [ ]:
from packages.Databases import DatabaseConnector, EntityOptions, EntitiesValuesFunctions
from datetime import datetime
import random
import string
import math
import time

class DataSeeder:
    def __init__(self, _entitiesValuesFunctions):
        self.entitiesValues = _entitiesValuesFunctions

    def generate_new_entity(self):
        existingEntities = [i[0] for i in self.entitiesValues.get_existing_entities()]
        for _ in range(1000000):
            newEntity = ''.join(random.choice(string.ascii_letters) for i in range(5)).upper()
            if newEntity not in existingEntities:
                break
        
        entityType = random.choice(list(EntityOptions)).value
        constants = []
        for _ in range(0,3):
            constants.append(float(random.randrange(0, 10000))/100)
            
        self.entitiesValues.add_entity(newEntity, entityType, constants[0], constants[1], constants[2])

    def generate_value(self, count, entityType, firstConstant, secondConstant, thirdConstant):
        if entityType == EntityOptions.SGFB.value:
            if random.randrange(0,10) <= 7:
                return firstConstant*count + secondConstant*count + thirdConstant
            else:
                return (firstConstant*count)**-2 + (secondConstant*count)**-3 + (thirdConstant*count)**-4
        elif entityType == EntityOptions.V.value:
            return abs(firstConstant*math.sin(count)) + secondConstant*random.uniform(-1,1) + thirdConstant*random.randrange(0,2)
        elif entityType == EntityOptions.FD.value:
            return firstConstant*secondConstant/(count + 1) + thirdConstant*random.randrange(0,2)
        elif entityType == EntityOptions.FR.value:    
            return abs(firstConstant*count + secondConstant*count + thirdConstant*random.uniform(-1,1))

    def add_entity_value(self, count, code):
        entityDetails = self.entitiesValues.get_entity_details(code)[0]
        value = self.generate_value(count, 
                                    entityDetails[1], 
                                    float(entityDetails[2]), 
                                    float(entityDetails[3]), 
                                    float(entityDetails[4]))
        self.entitiesValues.add_entity_value(code, datetime.now(), value)

    def run(self):
        for _ in range(0,10):
            self.generate_new_entity()
        entities = [i[0] for i in self.entitiesValues.get_existing_entities()]
        i=0
        while True:
            i+=1
            for entity in entities:
                self.add_entity_value(i, entity)
            time.sleep(random.randrange(0,5))

if __name__ = "__main__":
    with DatabaseConnector('EntitiesAndValues', 'data_seeder') as conn:
        _entitiesValues = EntitiesValuesFunctions(conn)
        _dataSeeder = DataSeeder(_entitiesValues)
        _dataSeeder.run()

In [ ]:
from packages.Databases import DatabaseConnector, EntityOptions, EntitiesValuesFunctions
from flask import Flask, jsonify, request
from datetime import datetime

def ValuesTrackerApi(_entitiesValues):
    app = Flask(__name__)
    app.config['EntitiesValues'] = _entitiesValues
    return app

@app.route('/api/get_existing_entities', methods=['GET'])
def get_entities():
    existingEntities = [i[0] for i in app.config['EntitiesValues'].get_existing_entities()]
    return jsonify(existingEntities)
    
@app.route('/api/get_historical_values', methods=['GET'])
def get_historical_values():
    code = request.args.get('code')
    historicalValues = [{'value': i[2], 'timestamp': i[1]} for i in app.config['EntitiesValues'].get_values(code, datetime.min)]
    return jsonify(historicalValues)

@app.route('/api/get_new_values', methods=['GET'])
def get_new_values():
    code - request.args.get('code')
    timestamp = request.args.get('timestamp')
    newValues = [{'value': i[2], 'timestamp': i[1]} for i in app.config['EntitiesValues'].get_values(code, timestamp)]
    return jsonify(newValues)

if __name__ == "__main__":
    with DatabaseConnector('EntitiesAndValues', 'api', 'db', 5432) as conn:
        _entitiesValues = EntitiesValuesFunctions(conn)
        app = ValuesTrackerApi(_entitiesValues)
        app.run()